In [15]:
pip install transformers

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/Anthony-Alridge/QualityEstimation
%cd 'QualityEstimation'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Cloning into 'QualityEstimation'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 113 (delta 50), reused 88 (delta 27), pack-reused 0
Receiving objects: 100% (113/113), 16.61 MiB | 7.63 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/QualityEstimation/QualityEstimation/QualityEstimation


In [0]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from torch.utils import data

In [0]:
f = open('data/en-zh/dev.enzh.src') # Open file on read mode
lines_en = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/dev.enzh.mt') # Open file on read mode
lines_de = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/dev.enzh.scores') # Open file on read mode
scores = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

# print(lines_de)

In [19]:
path = '/content/gdrive/My Drive/NLP Chinese Models/distilbert/'
model = ppb.DistilBertForSequenceClassification.from_pretrained(path)  # re-load
tokenizer = ppb.DistilBertTokenizer.from_pretrained(path)  # re-load
model.cuda()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [0]:
scores_ls = []
for s in scores:
  if s is None or s is '':break
  scores_ls.append(float(s))

labels = torch.FloatTensor(scores_ls)

datas = []
for i,j in zip(lines_en, lines_de):
  if i is '':break
  datas.append(tokenizer.encode(i,text_pair=j,max_length=80,pad_to_max_length=True))
data_tensor = torch.LongTensor(datas)

my_dataset = data.TensorDataset(data_tensor, labels) # create your datset
my_dataloader = data.DataLoader(my_dataset,batch_size=32,shuffle=False) 

In [0]:
model.train()
device=torch.device('cuda')
dtype = torch.long
outs = []
for t, (x, y) in enumerate(my_dataloader):
            # model.train()  # put model to training mode
            with torch.no_grad():
              x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
              y = y.to(device=device, dtype=torch.float)
              outputs = model(x)
              outputs = torch.flatten(outputs[0]).cpu()
              outputs = outputs.numpy()
              outs.append(outputs)
              # outs.append(torch.flatten(outputs))
# out_flat = torch.flatten(outs)
# outs = outs.flatten()
# print(outs)

with open('/content/gdrive/My Drive/NLP Chinese Models/distil_predictions.txt', 'a') as f:
    for a in outs:
      for x in a:
        print(x, file=f)
        